# YouTube Video Transcription Pipeline

In [1]:
pip install google-api-python-client

DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.11/site-packages/pdfservices_sdk-4.0.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install openai-whisper


DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.11/site-packages/pdfservices_sdk-4.0.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install yt-dlp

DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.11/site-packages/pdfservices_sdk-4.0.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


In [5]:
from googleapiclient.discovery import build  # Import YouTube Data API client
import os
import subprocess
import whisper
import json
import warnings

# Function to dynamically set API key
def set_api_key():
    import getpass
    return getpass.getpass("Enter your YouTube Data API key: ")

# Function to fetch one video from a playlist
def get_single_playlist_video(playlist_id, youtube_api_key):
    youtube = build("youtube", "v3", developerKey=youtube_api_key)
    response = youtube.playlistItems().list(
        playlistId=playlist_id,
        part="snippet",
        maxResults=1  # Fetch only 1 video
    ).execute()

    if "items" in response and len(response["items"]) > 0:
        video_id = response["items"][0]["snippet"]["resourceId"]["videoId"]
        video_title = response["items"][0]["snippet"]["title"]
        print(f"Selected playlist video: {video_title} (ID: {video_id})")
        return {"id": video_id, "title": video_title}
    else:
        raise Exception("No videos found in the playlist.")

def download_video(video_id, output_path="video"):
    video_url = f"https://www.youtube.com/watch?v={video_id}"
    # Use quotes around the output format string to avoid shell parsing errors
    command = f"yt-dlp -o \"{output_path}.%(ext)s\" {video_url}"
    try:
        print(f"Downloading video: {video_url}")
        subprocess.run(command, shell=True, check=True)

        # Find the downloaded file
        actual_file = next((f for f in os.listdir() if f.startswith(output_path) and not f.endswith('.part')), None)
        if actual_file:
            print(f"Video downloaded: {actual_file}")
            return actual_file
        else:
            raise FileNotFoundError("Downloaded video file not found.")
    except subprocess.CalledProcessError as e:
        print(f"Error downloading video: {e}")
        raise

# Function to extract audio from a video
def extract_audio(video_path, output_audio_path="audio.mp3"):
    command = f"ffmpeg -y -i \"{video_path}\" -q:a 0 -map a \"{output_audio_path}\""
    try:
        subprocess.run(command, shell=True, check=True)
        print(f"Audio extracted: {output_audio_path}")
    except subprocess.CalledProcessError as e:
        print(f"Error extracting audio: {e}")
        raise

# Function to transcribe audio using Whisper
def transcribe_audio(audio_path, model_name="base"):
    print(f"Loading Whisper model ({model_name})...")
    model = whisper.load_model(model_name)
    result = model.transcribe(audio_path)
    print("Transcription completed.")
    return result["text"]

# Interactive execution
YOUTUBE_API_KEY = set_api_key()
TEST_PLAYLIST_ID = "PL0mJAd4U08EX7lnT5oHHClA1ReoTmiVWy"

try:
    print("\nProcessing a video from the playlist...")
    playlist_video = get_single_playlist_video(TEST_PLAYLIST_ID, YOUTUBE_API_KEY)
    video_id = playlist_video["id"]
    video_title = playlist_video["title"]
    print(f"Processing video from playlist: {video_title}")

    downloaded_video_file = download_video(video_id)
    print(f"Downloaded video file: {downloaded_video_file}")

    audio_file = f"{video_id}_audio.mp3"
    extract_audio(downloaded_video_file, audio_file)
    print(f"Extracted audio file: {audio_file}")

    transcript = transcribe_audio(audio_file, model_name="base")
except Exception as e:
    print(f"An error occurred: {e}")


Enter your YouTube Data API key:  ········



Processing a video from the playlist...
Selected playlist video: Insights 2021: How Kraft Heinz is Innovating Market Research (ID: MisyrAMeRuM)
Processing video from playlist: Insights 2021: How Kraft Heinz is Innovating Market Research
[youtube] Extracting URL: https://www.youtube.com/watch?v=MisyrAMeRuM
[youtube] MisyrAMeRuM: Downloading webpage
[youtube] MisyrAMeRuM: Downloading ios player API JSON
[youtube] MisyrAMeRuM: Downloading mweb player API JSON
[youtube] MisyrAMeRuM: Downloading m3u8 information
[info] MisyrAMeRuM: Downloading 1 format(s): 137+251
[download] video.mkv has already been downloaded
Video downloaded: video.mkv
Downloaded video file: video.mkv


ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1_3 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --e

Audio extracted: MisyrAMeRuM_audio.mp3
Extracted audio file: MisyrAMeRuM_audio.mp3
Loading Whisper model (base)...


/opt/anaconda3/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription completed.
